In [1]:
import pandas as pd

# Construct file path
file_path = "data_files/Quotations.xlsx"

# Read xlsx file into pandas DataFrame
quots_df = pd.read_excel(file_path)

In [3]:
# Construct file path
file_path_customers = "data_files/Customers.xlsx"

# Read xlsx file into pandas DataFrame
customers_df = pd.read_excel(file_path_customers)

In [27]:
quots_df_light = quots_df.loc[:, ['Quotation No.', 'Sales Document No.', 'MS code', 'Cumulative order quantity in sales units', 'Sold-to party','Name: Sold-to party', 'First Date (First Line)','Document Date (date received/sent)','BU']]
quots_df_light['year_month'] = quots_df_light['Document Date (date received/sent)'].dt.year.astype(str) + quots_df_light['Document Date (date received/sent)'].dt.month.apply(lambda x: f'{x:02}')

# extract year and month from year_month column
quots_df_light['year'] = quots_df_light['year_month'].str[:4].astype(int)
quots_df_light['month'] = quots_df_light['year_month'].str[4:].astype(int)

# calculate fiscal year based on month value
quots_df_light['fiscal_year'] = (quots_df_light['year'] + ((quots_df_light['month'] >= 4) * 1) - 1).astype(str).str[-2:]
quots_df_light['fiscal_year'] = 'FY' + quots_df_light['fiscal_year'].astype(str)

quots_df_light = quots_df_light.loc[:, ['Quotation No.', 'Sales Document No.', 'Sold-to party','Name: Sold-to party', 'fiscal_year']]

In [29]:
quots_df_light = quots_df_light.rename(columns={'Sold-to party':'sold_to_customer'})

In [30]:
customers_df['sold_to_customer'] = customers_df['sold_to_customer'].astype(str)
quots_df_light['sold_to_customer'] = quots_df_light['sold_to_customer'].str.strip()
customers_df['sold_to_customer'] = customers_df['sold_to_customer'].astype(str)
quots_df_light['sold_to_customer'] = quots_df_light['sold_to_customer'].str.strip()

In [31]:
merged_df = quots_df_light.merge(customers_df[['sold_to_customer', 'customer_name', 'indirect_direct', 'channel','company_code_n']], on='sold_to_customer',how='left')

In [32]:
count_df = merged_df.groupby(['fiscal_year', 'sold_to_customer']).agg({'Quotation No.': 'nunique', 'Sales Document No.': 'nunique'})

In [35]:
unique_so = merged_df.groupby(['fiscal_year', 'sold_to_customer'])['Sales Document No.'].nunique(dropna='True').reset_index()
unique_so = unique_so.rename(columns={'Sales Document No.': 'SO qty'})
unique_quote = merged_df.groupby(['fiscal_year', 'sold_to_customer'])['Quotation No.'].nunique(dropna='True').reset_index()
unique_quote = unique_quote.rename(columns={'Quotation No.': 'Quote qty'})

In [36]:
merged_df = merged_df.merge(unique_so, on='sold_to_customer',how='left')
merged_df = merged_df.merge(unique_quote, on='sold_to_customer',how='left')

In [22]:
# Create an ExcelWriter object
writer = pd.ExcelWriter('data_files/temp.xlsx')
# Save each DataFrame to a separate sheet in the same file
merged_df.to_excel(writer, sheet_name='temp')
# Save the file
writer.close()

In [40]:
# Create an ExcelWriter object
writer = pd.ExcelWriter('data_files/temp.xlsx')
# Save each DataFrame to a separate sheet in the same file
unique_so.to_excel(writer, sheet_name='unique_so')
unique_quote.to_excel(writer, sheet_name='unique_quote')
# Save the file
writer.close()

In [42]:
# Construct file path
file_path_orders = "data_files/Orders.xlsx"

# Read xlsx file into pandas DataFrame
orders_df = pd.read_excel(file_path_orders)

In [43]:
# Construct file path
file_path_quots = "data_files/Quotations_t.xlsx"

# Read xlsx file into pandas DataFrame
second_quots_df = pd.read_excel(file_path_quots)

In [50]:
orders_df_light = orders_df.loc[:, ['company_code_n', 'year_month', 'sales_order_so', 'sold_to_customer']]

# extract year and month from year_month column
orders_df_light['year_month'] = orders_df_light['year_month'].astype(str)
orders_df_light['year'] = orders_df_light['year_month'].str[:4].astype(int)
orders_df_light['month'] = orders_df_light['year_month'].str[4:].astype(int)

# calculate fiscal year based on month value
orders_df_light['fiscal_year'] = (orders_df_light['year'] + ((orders_df_light['month'] >= 4) * 1) - 1).astype(str).str[-2:]
orders_df_light['fiscal_year'] = 'FY' + orders_df_light['fiscal_year'].astype(str)

orders_df_light = orders_df_light.loc[:, ['fiscal_year', 'company_code_n', 'sales_order_so', 'sold_to_customer']]

In [56]:
second_quots_df['so_created_no'] = second_quots_df['so_created_no'].fillna(0).astype(int)

In [60]:
second_quots_df = second_quots_df.loc[:, ['quotation_no', 'Year of quotation_date', 'so_created_no', 'sold_to_customer']]

In [ ]:
# Drop duplicate values in column A and create new dataframe with columns A and B
new_df = second_quots_df.drop_duplicates()
new_df['sold_to_customer'] = new_df['sold_to_customer'].astype(str)
new_df['sold_to_customer'] = new_df['sold_to_customer'].str.strip()
new_df['one'] = 1

customers_df['sold_to_customer'] = customers_df['sold_to_customer'].astype(str)
customers_df['sold_to_customer'] = customers_df['sold_to_customer'].str.strip()

In [90]:
# Select rows from df1 where column C values match column A values in df2
new_df = new_df[new_df['sold_to_customer'].isin(customers_df['sold_to_customer'])]

In [93]:
# Create an ExcelWriter object
writer = pd.ExcelWriter('data_files/temp.xlsx')
# Save each DataFrame to a separate sheet in the same file
new_df.to_excel(writer, sheet_name='temp')
# Save the file
writer.close()

In [ ]:
new_df = second_quots_df.drop_duplicates()
new_df['sold_to_customer'] = new_df['sold_to_customer'].astype(str)
new_df['sold_to_customer'] = new_df['sold_to_customer'].str.strip()

customers_df['sold_to_customer'] = customers_df['sold_to_customer'].astype(str)
customers_df['sold_to_customer'] = customers_df['sold_to_customer'].str.strip()

result_df = new_df[~new_df['sold_to_customer'].isin(customers_df['sold_to_customer'])]

# Create an ExcelWriter object
writer = pd.ExcelWriter('data_files/temp2.xlsx')
# Save each DataFrame to a separate sheet in the same file
result_df.to_excel(writer, sheet_name='temp')
# Save the file
writer.close()

In [ ]:
all_so = orders_df_light.drop_duplicates(subset=['sales_order_so'],keep='first')
print(len(all_so))
all_so['sales_order_so'] = all_so['sales_order_so'].astype(str)
all_so = orders_df_light.drop_duplicates(subset=['sales_order_so'],keep='first')
all_so['sales_order_so'] = all_so['sales_order_so'].astype(str)
print(len(all_so))

quots_help = second_quots_df[['so_created_no', 'quotation_no']]
quots_help = quots_help.drop_duplicates(subset=['so_created_no'],keep='first')
quots_help = quots_help.drop(quots_help[quots_help['so_created_no'] == 0].index)
quots_help['so_created_no'] = quots_help['so_created_no'].astype(str)
quots_help = quots_help.rename(columns={'so_created_no': 'sales_order_so'})

In [151]:
merge_all_so = all_so.merge(quots_help, on='sales_order_so', how='left')

In [154]:
# Create an ExcelWriter object
writer = pd.ExcelWriter('data_files/temp3.xlsx')
# Save each DataFrame to a separate sheet in the same file
merge_all_so.to_excel(writer, sheet_name='temp3')
# Save the file
writer.close()

In [ ]:
result_df = merge_all_so['quotation_no'].isna().groupby(merge_all_so['fiscal_year']).sum().astype(int)
print(result_df)

In [159]:
second_quots_df['counts'] = 1

In [163]:
second_quots_df_no_dubl = second_quots_df.drop_duplicates(subset=['quotation_no'],keep='first')

In [164]:
quots_count_df = second_quots_df_no_dubl['counts'].groupby(second_quots_df_no_dubl['Year of quotation_date']).sum().astype(int)

In [166]:
second_quots_df_no_dubl_so = second_quots_df.drop_duplicates(subset=['so_created_no'],keep='first')

In [168]:
second_quots_df_no_dubl_so = second_quots_df_no_dubl_so[second_quots_df_no_dubl_so['so_created_no'] != 0]

In [170]:
so_count_df = second_quots_df_no_dubl_so['counts'].groupby(second_quots_df_no_dubl_so['Year of quotation_date']).sum().astype(int)

In [ ]:
all_so['counts'] = 1
so_count_df_full = all_so['counts'].groupby(all_so['fiscal_year']).sum().astype(int)